# 作業

### 嘗試用 keras 的 DepthwiseConv2D 等 layers 實做 Inverted Residual Block.
   - depthwise's filter shape 爲 (3,3), padding = same
   - 不需要給 alpha, depth multiplier 參數
   - expansion 因子爲 6

##### 載入套件

In [1]:
import warnings
warnings.filterwarnings('ignore')

from keras.models import Input, Model
from keras.layers import DepthwiseConv2D, Conv2D, BatchNormalization, ReLU, Add

Using TensorFlow backend.


#### 原始的Residual Block是先降維->Conv->升維->相加
#### Inverted Residual Block是先升維->Conv->降維->相加 (多channels減少information loss)
<img src="Inverted_residuals.png" width="400">


#### Bottleneck最後輸出用Linear反而比ReLU效果好
#### Bottleneck 與 Bottleneck相互連結效果最佳
<img src="Ablation.png" width="400">

##### 定義 Separable Convolution 函數 (請在此實做)

In [2]:
from keras import backend as K

def InvertedRes(input, expansion):

    # Expansion Layer
    x = Conv2D(expansion*K.int_shape(input)[-1], (1,1), padding='same')(input)
    x = BatchNormalization()(x)
    x = ReLU(max_value=6)(x)
    
    # Depthwise Convolution (bias可有可無)
    x = DepthwiseConv2D((3,3), padding='same', depth_multiplier=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU(max_value=6)(x)
    
    # Pointwise Convolution
    x = Conv2D(3, (1,1))(x)
    x = BatchNormalization()(x)
    #x = ReLU()(x)
    
    # Add Layer
    added = Add()([input, x])
    
    return added

##### 建構模型

In [3]:
input = Input((64, 64, 3))
output = InvertedRes(input, 6)
model = Model(inputs=input, outputs=output)
model.summary()







Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 18)   72          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 18)   72          conv2d_1[0][0]                   
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 64, 64, 18)   0           batch_normalization_1[0][0]      
______________________________________________________________________________________

<img src="MobilenetV2.png" width="400">